## Helper Functions

In [1]:
from Helper_functions import Get_valid_chunk_sizes
from Helper_functions import Build_canonical_map
from Helper_functions import Get_seed

In [2]:
from itertools import product 
from Bio import SeqIO
import numpy as np
from scipy.interpolate import PchipInterpolator
from pathlib import Path
import joblib

In [3]:
def Generate_arrays(Valid_chunk_sizes, Kmer_size, Canonical_map, Chromosome_seq, Chromosome_id, Num_chunks):
    
    # generate a kmer --> index for fast numpy indexing 
    All_kmers = sorted(set(Canonical_map.values()))
    Kmer_index = {kmer : index for index, kmer in enumerate(All_kmers)}

    Counts_list = []
    Cov_list = []

    for Chunk_size in Valid_chunk_sizes:
        #Numpy array for storing kmers counts at that given chunk_size 
        Counted_chunks_array = np.zeros((Num_chunks, len(All_kmers)), dtype="int64") # shape = (N_chunks, N_kmers)


        Half_chunk_size = Chunk_size // 2
        for Chunk in range(Num_chunks):
            # generate a random seed location to cut the chromosome at 
            Seed_location = Get_seed(Chromosome_id, Half_chunk_size, Chunk, len(Chromosome_seq), Interpolator = True)
            # Count the kmers across that chunk, use numpy indexing (fast) to input into array 
            Start, Stop = int(Seed_location - Half_chunk_size), int(Seed_location + Half_chunk_size - Kmer_size + 1)
            for Window in range(Start, Stop):
                Raw_kmer = Chromosome_seq[Window : (Window + Kmer_size)]
                Canonical_kmer = Canonical_map.get(Raw_kmer)
                # Input into array 
                col = Kmer_index[Canonical_kmer]
                Counted_chunks_array[Chunk, col] += 1  
                
        # Calculte the mean kmer counts across all of the chunks 
        Mean_count = Counted_chunks_array.mean(axis = 0)
        # add to List 
        Counts_list.append(Mean_count)
        
        # Calculate the Cov --> Creates a 136 by 136 matrix 
        Mean_cov = np.cov(Counted_chunks_array, rowvar = False)
        # add to list
        Cov_list.append(Mean_cov)
    
    return Counts_list, Cov_list

In [4]:


def Generate_counts_interpolator(Mean_counts_list, Valid_chunk_sizes):
    #convert into array 
    chunk_sizes_array = np.array(Valid_chunk_sizes)
    mean_vecs_array = np.asarray(Mean_counts_list) # Shape (n_chunk_sizes, n_kmers)
    n_chunk_sizes, n_kmers = mean_vecs_array.shape

    mean_vec_interpolators = []

    # iterate through each of the kmers 
    for kmer in range(n_kmers):
        # get all of the values for that kmer across all of the arrays 
        values_at_kmer = mean_vecs_array[:, kmer] 
        # interpolate across those values 
        interpolator = PchipInterpolator(chunk_sizes_array, values_at_kmer, extrapolate = False)
        # Add interpolator function into list 
        mean_vec_interpolators.append(interpolator)
   
    return mean_vec_interpolators 
    

def Generate_cov_interpolator(Mean_cov_list, Valid_chunk_sizes):

    # convert into array 
    cov_vecs_array = np.asarray(Mean_cov_list) # shape = (10, 136, 136)
    chunk_sizes_array = np.array(Valid_chunk_sizes)

    n_chunk_sizes, kmer_index, kmer_columns = cov_vecs_array.shape

    cov_interpolators = []

    for kmer1 in range(kmer_index):
        row_interpolators = []
        for kmer2 in range(kmer_columns):
            # retrieve the values of kmer 1 + kmer 2 pairings across all of the arrays 
            values_at_pairing = cov_vecs_array[:, kmer1, kmer2]
            interpolator = PchipInterpolator(chunk_sizes_array, values_at_pairing, extrapolate = False)
            row_interpolators.append(interpolator)
        cov_interpolators.append(row_interpolators)
    return cov_interpolators


def Export_Interpolators(Mean_counts_interpolator, Cov_interpolator, Valid_chunk_sizes, Kmer_size, Chromosome_id, Genome_name, Output_dir):

    Output_path = Path(Output_dir, Genome_name)
    Output_path.mkdir(parents = True, exist_ok = True)
    Output_file = Output_path / f"{Chromosome_id}.pkl"

    Bundle = {
        "Mean_counts_interpolator" : Mean_counts_interpolator,
        "Cov_interpolator" : Cov_interpolator,
        "Chunk_sizes" : Valid_chunk_sizes,
        "Kmer_size" : Kmer_size,
        "Meta_data" : {"Genome_name" : Genome_name, "Chromosome_id" : Chromosome_id}
                        
    }
    joblib.dump(Bundle, Output_file)


In [13]:
%%time

def Master_Interpolator_Generation(Genome_FNA_path, Kmer_size, Num_chunks, Output_dir): 
    """
    Takes a raw FNA file, and creates a .pkl file.
    Pkl contains the equations for getting the expected "mean_kmer_count",
    and the expected "Cov_matrix".
    Creates a seperate file for each kmer_size 
    """
    # Get the Genome name from the Genome_FNA_path 
    # Chromosome.description appears too variable 

    Genome_name_ext = Path(Genome_FNA_path).name

    Genome_name = ".".join(Genome_name_ext.split(".")[:-1])

    # Builds a fast look up system for finding the canonical kmer (lexographical)
    Canonical_map = Build_canonical_map(Kmer_size) 

    print(f"Processing Genome : {Genome_name}")
    for Chromosome in SeqIO.parse(Genome_FNA_path, "fasta"):
        print(f"Processing Chromosome : {Chromosome.id}")
        # A list (in order of smallest to largest) of valid chunk_sizes for the given chromosome
        Valid_chunk_sizes = Get_valid_chunk_sizes(len(Chromosome.seq))
        
        # returns an np.array of the mean kmer counts across all chunk sizes (shape of : n_chunk_sizes, n_kmers). 
        #And the associated covariance matrix (shape of : n_chunk_sizes, n_kmers, n_kmers)
        Mean_counts_list, Mean_cov_list = Generate_arrays(Valid_chunk_sizes, Kmer_size, Canonical_map, Chromosome.seq, Chromosome.id, Num_chunks)
    
        # A list of interpolator functions for each kmer in the Mean_counts_array
        Mean_counts_interpolator = Generate_counts_interpolator(Mean_counts_list, Valid_chunk_sizes)
        # A matrix of interpolator functins for each pair of kmers in the mean_cov_array 
        Cov_interpolator = Generate_cov_interpolator(Mean_cov_list, Valid_chunk_sizes)

        # Save the interpolators
        print(f"Saving Chromosome : {Chromosome.id}")
        Export_Interpolators(Mean_counts_interpolator, Cov_interpolator, Valid_chunk_sizes, Kmer_size, Chromosome.id, Genome_name, Output_dir)




Num_chunks = 150
Kmer_size = 4
Output_dir = "C:/users/henry chapman/Documents/Galaxy/Output/Pipe_1/Interpolators/Kmer_size_4/Raw_counts"

for Genome_FNA_path in Path("C:/users/henry chapman/Documents/Galaxy/Raw_data/Genomes").iterdir():
    if ".ipynb_checkpoints" in Genome_FNA_path.name:
        continue
    Master_Interpolator_Generation(Genome_FNA_path, 
                               Kmer_size, 
                               Num_chunks, 
                               Output_dir)
    
    

Processing Genome : 1Deinococcus_radiodurans
Processing Chromosome : NZ_CP038663.1
Saving Chromosome : NZ_CP038663.1
Processing Chromosome : NZ_CP038664.1
Saving Chromosome : NZ_CP038664.1
Processing Chromosome : NZ_CP038666.1
Saving Chromosome : NZ_CP038666.1
Processing Chromosome : NZ_CP038665.1
Saving Chromosome : NZ_CP038665.1
Processing Genome : Escherichia_coli_ASM584v2
Processing Chromosome : U00096.3
Saving Chromosome : U00096.3
Processing Genome : GCA_006094375.1_ASM609437v1_genomic
Processing Chromosome : CP035288.1
Saving Chromosome : CP035288.1
Processing Chromosome : CP035289.1
Saving Chromosome : CP035289.1
Processing Chromosome : CP035290.1
Saving Chromosome : CP035290.1
Processing Genome : GCF_022354085.1_ASM2235408v1_genomic
Processing Chromosome : NZ_CP055055.1
Saving Chromosome : NZ_CP055055.1
Processing Chromosome : NZ_CP055056.1
Saving Chromosome : NZ_CP055056.1
CPU times: total: 10min 56s
Wall time: 6min 18s
